In [ ]:
# scripts/stock_price_analysis.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime
import os

In [ ]:
# config
sns.set(style="darkgrid")
plt.rcParams['figure.figsize'] = (12,6)

In [ ]:
tickers = ["AAPL", "TSLA", "MSFT"]
start = "2019-01-01"
end = datetime.today().strftime("%Y-%m-%d")

In [ ]:
# create outputs folder if not exists
os.makedirs("outputs", exist_ok=True)

In [ ]:
# download data
df = yf.download(tickers, start=start, end=end)["Adj Close"]
if isinstance(df, pd.Series):
    df = df.to_frame()
df = df.dropna(how="all")

In [ ]:
# Price trends
ax = df.plot(title="Adjusted Close Price", linewidth=1.2)
ax.set_ylabel("Price (USD)")
ax.set_xlabel("Date")
plt.tight_layout()
plt.savefig("outputs/price_trends.png", dpi=150)
plt.close()

In [ ]:
# Returns
returns = df.pct_change().dropna()

In [ ]:
# Distribution plots
for t in returns.columns:
    plt.figure(figsize=(10,5))
    sns.histplot(returns[t], bins=80, kde=True)
    plt.title(f"{t} - Daily Return Distribution")
    plt.xlabel("Daily Return")
    plt.ylabel("Frequency")
    plt.tight_layout()
    plt.savefig(f"outputs/{t}_returns_dist.png", dpi=150)
    plt.close()

In [ ]:
# Rolling volatility
rolling_std = returns.rolling(window=30).std() * np.sqrt(252)
ax = rolling_std.plot(title="30-day Rolling Annualized Volatility")
ax.set_ylabel("Volatility")
plt.tight_layout()
plt.savefig("outputs/rolling_volatility.png", dpi=150)
plt.close()

In [ ]:
# Moving averages for AAPL
ma50 = df.rolling(window=50).mean()
ma200 = df.rolling(window=200).mean()

In [ ]:
plt.figure(figsize=(14,7))
plt.plot(df.index, df["AAPL"], label="AAPL Price")
plt.plot(ma50.index, ma50["AAPL"], label="50-day MA")
plt.plot(ma200.index, ma200["AAPL"], label="200-day MA")
plt.title("AAPL Price & Moving Averages")
plt.legend()
plt.tight_layout()
plt.savefig("outputs/AAPL_moving_averages.png", dpi=150)
plt.close()

In [ ]:
# Correlation heatmap
corr = returns.corr()
plt.figure(figsize=(6,5))
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Correlation of Daily Returns")
plt.tight_layout()
plt.savefig("outputs/returns_corr_heatmap.png", dpi=150)
plt.close()

In [ ]:
# Print basic stats
print("Mean daily returns:\n", returns.mean())
print("\nAnnualized mean returns:\n", returns.mean()*252)
print("\nAnnualized volatility:\n", returns.std()*np.sqrt(252))